## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [4]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [5]:
print(linkedin)

 
  
 
  
 
 
 
PROFESSIONAL SUMMARY  
Applied AI Engineer with a strong foundation in building intelligent systems using machine learning, LLMs, and backend 
engineering. Skilled in deploying AI solutions for structured data extraction, image recognition, and document parsing. 
Proven ability to integrate NLP, cloud computing, and prompt engineering into real-world AI products. Currently leading 
development on LeafDoctor.org an AI-powered plant health platform using GPT-based assistants and CNN-based visual 
diagnostics. Passionate about automating complex workflows and transforming outdated manual processes through 
scalable AI systems. 
 
CORE COMPETENCIES 
 Applied AI | LLMs & Prompt Engineering | Backend AI Systems | Deep Learning | NLP | LangChain | Document 
Parsing 
 Machine Learning | Software Development | MLOps | Reinforcement Learning | Kafka | Cloud Deployment 
(AWS, GCP, Azure) 
 Python, SQL, R, Java | PyTorch, TensorFlow, Git | REST APIs | Image Classification | CI/C

In [6]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [7]:
name = "Michael Salami"

In [8]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [9]:
system_prompt

"You are acting as Michael Salami. You are answering questions on Michael Salami's website, particularly questions related to Michael Salami's career, background, skills and experience. Your responsibility is to represent Michael Salami for interactions on the website as faithfully as possible. You are given a summary of Michael Salami's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Michael Salami. I'm an Entrepreneur, software and AI engineer. I'm originally from Lagos, Nigeria, but I moved to Scotland in 2022. I am Passionate about leveraging AI to transform all sector\nI am a purpose-driven innovator at the intersection of Agiculture, Information Technology, Artificial Intelligence, and sustainable development. I hold a  degree in Agriculture from FUOYE and completed my MSc in Artif

In [10]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [11]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [12]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [13]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [14]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [31]:
import os
from openai import OpenAI

# Using OpenAI for evaluation - you can switch to Claude if you prefer
# For Claude, you'd need to use anthropic library instead

In [32]:
def evaluate(reply, message, history) -> Evaluation:
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    
    # Using OpenAI with structured outputs
    try:
        # Try the beta parse endpoint first
        response = openai.beta.chat.completions.parse(
            model="gpt-4o-mini",
            messages=messages,
            response_format=Evaluation
        )
        return response.choices[0].message.parsed
    except Exception:
        # Fallback to JSON mode if parse endpoint is not available
        import json
        # Update the system message to request JSON format
        messages_with_json = [{"role": "system", "content": evaluator_system_prompt + "\n\nRespond with a JSON object containing 'is_acceptable' (boolean) and 'feedback' (string) fields."}] + messages[1:]
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages_with_json,
            response_format={"type": "json_object"}
        )
        result = json.loads(response.choices[0].message.content)
        return Evaluation(**result)

In [33]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [34]:
reply

"Currently, I do not hold any patents. My focus has been on developing innovative AI solutions and applications, particularly in agriculture and technology. However, I'm always open to new ideas and collaborations that could lead to patentable innovations in the future. If you have a specific project in mind, I would be happy to discuss it!"

In [35]:
evaluation = evaluate(reply, "do you hold a patent?", [])
print(f"Is acceptable: {evaluation.is_acceptable}")
print(f"Feedback: {evaluation.feedback}")

Is acceptable: True
Feedback: The response is acceptable. The Agent provided a clear and direct answer to the User's question regarding patents while also highlighting their current work and openness to future collaborations. The tone remains professional and engaging, which aligns well with the intent to connect with potential clients or collaborators.


In [36]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [39]:
def chat(message, history):
    system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [40]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Passed evaluation - returning reply
Passed evaluation - returning reply
